In [1]:
%cd BraTS2020/

c:\Users\buihu\OneDrive\Tài liệu\AIO\Research\ACCV\Diff-UNet-main\BraTS2020


In [2]:
!python train.py

Traceback (most recent call last):
  File "train.py", line 6, in <module>
    from light_training.evaluation.metric import dice
  File "c:\Users\buihu\OneDrive\Tài liệu\AIO\Research\ACCV\Diff-UNet-main\BraTS2020\light_training\evaluation\metric.py", line 16, in <module>
    from medpy import metric
ModuleNotFoundError: No module named 'medpy'


In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, in_dim, heads=8):
        super().__init__()
        self.heads = heads
        self.scale = (in_dim // heads) ** -0.5

        self.to_qkv = nn.Linear(in_dim, in_dim * 3, bias=False)
        self.to_out = nn.Linear(in_dim, in_dim)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.to_qkv(x).chunk(3, dim=-1)
        q, k, v = map(lambda t: t.reshape(B, N, self.heads, C // self.heads).transpose(1, 2), qkv)

        dots = (q @ k.transpose(-1, -2)) * self.scale
        attn = dots.softmax(dim=-1)

        out = (attn @ v).transpose(1, 2).reshape(B, N, C)
        return self.to_out(out)

In [ ]:
class BasicUNetDe(nn.Module):
    # ...
    def __init__(
        self,
        spatial_dims: int = 3,
        in_channels: int = 1,
        out_channels: int = 2,
        features: Sequence[int] = (32, 32, 64, 128, 256, 32),
        act: Union[str, tuple] = ("LeakyReLU", {"negative_slope": 0.1, "inplace": True}),
        norm: Union[str, tuple] = ("instance", {"affine": True}),
        bias: bool = True,
        dropout: Union[float, tuple] = 0.0,
        upsample: str = "deconv",
        dimensions: Optional[int] = None,
    ):
        # ...
        self.attention_0 = SelfAttention(features[0])
        self.attention_1 = SelfAttention(features[1])
        self.attention_2 = SelfAttention(features[2])
        self.attention_3 = SelfAttention(features[3])
        self.attention_4 = SelfAttention(features[4])

    def forward(self, x: torch.Tensor, t, embeddings=None, image=None):
        # ...
        x0 = self.conv_0(x, temb)
        x0 = self.attention_0(x0)
        if embeddings is not None:
            x0 += embeddings[0]

        x1 = self.down_1(x0, temb)
        x1 = self.attention_1(x1)
        if embeddings is not None:
            x1 += embeddings[1]

        x2 = self.down_2(x1, temb)
        x2 = self.attention_2(x2)
        if embeddings is not None:
            x2 += embeddings[2]

        x3 = self.down_3(x2, temb)
        x3 = self.attention_3(x3)
        if embeddings is not None:
            x3 += embeddings[3]

        x4 = self.down_4(x3, temb)
        x4 = self.attention_4(x4)
        if embeddings is not None:
            x4 += embeddings[4]

        # ...